# PyCity Schools Analysis

* District Schools, apart from being from the lange size schools  group, they tendo to do worse in Math and Reading Scores that Charter schools
* Regarding the size of the school, it seems that this parameter is related to the performance in subjects scores (math and reading). This is because the data show that Small and Medium Size shcools do better than Large size schools.
* Finally, budget per student seem to have a negative association with all scores. The "High Budget per Student" group ($ 635.75 - 655.0) is the one with the lowest average scores in math and reading
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# Files to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [6]:
#####      District Summary
#Calculate the total number of schools
nSchools=len(data["School ID"].unique())
# Calculate the total number of students
nStudents=len(data["Student ID"].unique())
# Calculate the total budget
tBudget=pd.DataFrame(data.groupby("School ID").mean())["budget"].sum()
# Calculate the average math score
aMath=data["math_score"].mean()
# Calculate the average reading score
aReading=data["reading_score"].mean()
# Calculate the percentage of students with a passing math score (70 or greater)
prMath=(data[(data['math_score']>=70)]["Student ID"].count()/data["Student ID"].count())*100
# Calculate the percentage of students with a passing reading score (70 or greater)
prReading=(data[(data['reading_score']>=70)]["Student ID"].count()/data["Student ID"].count())*100
# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
oPR=(aMath+aReading)/2
# Create a dataframe to hold the above results
District_Sumarry=pd.DataFrame({"Total Schools":nSchools,
             "Total Students": nStudents,
             "Total Budget": tBudget,
             "Average Math Score":aMath,
             "Average Reading Score":aReading,
             "% Passing Math":prMath,
             "% Passing Reading": prReading,
             "% Overall Pasing Rate": oPR}, index=[0])
# Optional: give the displayed data cleaner formatting
District_Sumarry
District_Sumarry.style.format({'Total Schools': "{:,.0f}",
                               'Total Students':"{:,.0f}",
                               "Total Budget": "$ {:,.2f}",
                               "Average Math Score":"{:,.2f} pts.",
                               "Average Reading Score":"{:,.2f} pts.",
                               "% Passing Math":"{:.1f} %",
                               "% Passing Reading": "{:.1f} %",
                               "% Overall Pasing Rate": "{:.1f} %"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pasing Rate
0,15,"39,170","$ 24,649,428.00",78.99 pts.,81.88 pts.,75.0 %,85.8 %,80.4 %


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [7]:
####  School Summary
##   Create an overview table that summarizes key metrics about each school, including:
# School Name
# School Type
# Total Students
### Obtain data frame from groupby table with students count
SchoolSummary=pd.DataFrame(data.groupby(["School ID","school_name", "type"])["Student ID"].count()) .reset_index()
# Total School Budget
# Per Student Budget
# Average Math Score
# Average Reading Score
### Obtain data frame from groupby table with budget and scores average by school
SchoolSummary=SchoolSummary.merge(pd.DataFrame(data.groupby(["School ID","school_name", "type"])["budget","math_score", "reading_score", "size"].mean()), on="school_name", how="left")
SchoolSummary["Per Student Budget"]=SchoolSummary["budget"]/SchoolSummary["Student ID"]
# % Passing Math
# % Passing Reading
### Obtain data frames from groupby table with students with passing scores count
SchoolSummary=SchoolSummary.merge(pd.DataFrame(data[data["math_score"]>=70].groupby(["School ID","school_name", "type"])["Student ID"].count()), on="school_name", how="left")
SchoolSummary=SchoolSummary.merge(pd.DataFrame(data[data["reading_score"]>=70].groupby(["School ID","school_name", "type"])["Student ID"].count()), on="school_name", how="left")
SchoolSummary=SchoolSummary.rename(columns={"Student ID_x":"Total Students",
                                            "Student ID_y":"% Passing Math",
                                            "Student ID":"% Passing Reading",
                                            "budget": "Total Budget",
                                            "math_score": "Average Math Score",
                                            "reading_score": "Average Reading Score",
                                            "type": "School Type",
                                            "size": "Size"})
### Calculate the % of passing students by subject
SchoolSummary["% Passing Math"]=(SchoolSummary["% Passing Math"]/SchoolSummary["Total Students"])*100
SchoolSummary["% Passing Reading"]=(SchoolSummary["% Passing Reading"]/SchoolSummary["Total Students"])*100
# Overall Passing Rate (Average of the above two)
SchoolSummary["% Overall Passing Rate"]=(SchoolSummary["% Passing Math"]+SchoolSummary["% Passing Reading"])/2
# Create a dataframe to hold the above results. Use "School Name" as index for the df
SchoolSummary=SchoolSummary.set_index("school_name")
SchoolSummary

,School ID,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Size,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,,,
Huang High School,0,District,2917,1910635.0,76.629414,81.182722,2917.0,655.0,65.683922,81.316421,73.500171
Figueroa High School,1,District,2949,1884411.0,76.711767,81.158020,2949.0,639.0,65.988471,80.739234,73.363852
Shelton High School,2,Charter,1761,1056600.0,83.359455,83.725724,1761.0,600.0,93.867121,95.854628,94.860875
Hernandez High School,3,District,4635,3022020.0,77.289752,80.934412,4635.0,652.0,66.752967,80.862999,73.807983
Griffin High School,4,Charter,1468,917500.0,83.351499,83.816757,1468.0,625.0,93.392371,97.138965,95.265668
Wilson High School,5,Charter,2283,1319574.0,83.274201,83.989488,2283.0,578.0,93.867718,96.539641,95.203679
Cabrera High School,6,Charter,1858,1081356.0,83.061895,83.975780,1858.0,582.0,94.133477,97.039828,95.586652
Bailey High School,7,District,4976,3124928.0,77.048432,81.033963,4976.0,628.0,66.680064,81.933280,74.306672
Holden High School,8,Charter,427,248087.0,83.803279,83.814988,427.0,581.0,92.505855,96.252927,94.379391


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [8]:
#Sort and display the top five schools in overall passing rate
SchoolSummary = SchoolSummary.sort_values("% Overall Passing Rate", ascending=False)
SchoolSummary.head(5)

,School ID,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Size,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,,,
Cabrera High School,6,Charter,1858,1081356.0,83.061895,83.975780,1858.0,582.0,94.133477,97.039828,95.586652
Thomas High School,14,Charter,1635,1043130.0,83.418349,83.848930,1635.0,638.0,93.272171,97.308869,95.290520
Pena High School,9,Charter,962,585858.0,83.839917,84.044699,962.0,609.0,94.594595,95.945946,95.270270
Griffin High School,4,Charter,1468,917500.0,83.351499,83.816757,1468.0,625.0,93.392371,97.138965,95.265668
Wilson High School,5,Charter,2283,1319574.0,83.274201,83.989488,2283.0,578.0,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [9]:
#Sort and display the five worst-performing schools
SchoolSummary = SchoolSummary.sort_values("% Overall Passing Rate", ascending=True)
SchoolSummary.head(5)

,School ID,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Size,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,,,
Rodriguez High School,11,District,3999,2547363.0,76.842711,80.744686,3999.0,637.0,66.366592,80.220055,73.293323
Figueroa High School,1,District,2949,1884411.0,76.711767,81.158020,2949.0,639.0,65.988471,80.739234,73.363852
Huang High School,0,District,2917,1910635.0,76.629414,81.182722,2917.0,655.0,65.683922,81.316421,73.500171
Johnson High School,12,District,4761,3094650.0,77.072464,80.966394,4761.0,650.0,66.057551,81.222432,73.639992
Ford High School,13,District,2739,1763916.0,77.102592,80.746258,2739.0,644.0,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
###   Math Scores by Grade
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
# Create a pandas series for each grade.
# Group each series by school
# Combine the series into a dataframe
for gr in list(data["grade"].unique()):
    if(gr==list(data["grade"].unique())[0]):
        MathGrade=pd.DataFrame(data[data["grade"]==gr].groupby(["school_name"])["math_score"].mean())
        MathGrade=MathGrade.rename(columns={"math_score":gr})
    elif(gr!=list(data["grade"].unique())[0]):
        MathGrade=MathGrade.merge(pd.DataFrame(data[data["grade"]==gr].groupby(["school_name"])["math_score"].mean()).reset_index(), on="school_name", how="left")
        MathGrade=MathGrade.rename(columns={"math_score":gr})

# Optional: give the displayed data cleaner formatting
MathGrade=MathGrade[["school_name","9th", "10th", "11th","12th"]].set_index("school_name").style.format({"9th":"{:.2f} %",
                   "10th":"{:.2f} %",
                   "11th":"{:.2f} %",
                   "12th":"{:.2f} %"})
MathGrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08 %,77.00 %,77.52 %,76.49 %
Cabrera High School,83.09 %,83.15 %,82.77 %,83.28 %
Figueroa High School,76.40 %,76.54 %,76.88 %,77.15 %
Ford High School,77.36 %,77.67 %,76.92 %,76.18 %
Griffin High School,82.04 %,84.23 %,83.84 %,83.36 %
Hernandez High School,77.44 %,77.34 %,77.14 %,77.19 %
Holden High School,83.79 %,83.43 %,85.00 %,82.86 %
Huang High School,77.03 %,75.91 %,76.45 %,77.23 %
Johnson High School,77.19 %,76.69 %,77.49 %,76.86 %


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
###   Reading Scores by Grade
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
# Create a pandas series for each grade. Hint: use a conditional statement.
# Group each series by school
# Combine the series into a dataframe
for gr in list(data["grade"].unique()):
    if(gr==list(data["grade"].unique())[0]):
        ReadingGrade=pd.DataFrame(data[data["grade"]==gr].groupby(["school_name"])["reading_score"].mean())
        ReadingGrade=ReadingGrade.rename(columns={"reading_score":gr})
    elif(gr!=list(data["grade"].unique())[0]):
        ReadingGrade=ReadingGrade.merge(pd.DataFrame(data[data["grade"]==gr].groupby(["school_name"])["reading_score"].mean()).reset_index(), on="school_name", how="left")
        ReadingGrade=ReadingGrade.rename(columns={"reading_score":gr})
# Optional: give the displayed data cleaner formatting
ReadingGrade=ReadingGrade[["school_name","9th", "10th", "11th","12th"]].set_index("school_name").style.format({"9th":"{:.2f} %",
                   "10th":"{:.2f} %",
                   "11th":"{:.2f} %",
                   "12th":"{:.2f} %"})
ReadingGrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30 %,80.91 %,80.95 %,80.91 %
Cabrera High School,83.68 %,84.25 %,83.79 %,84.29 %
Figueroa High School,81.20 %,81.41 %,80.64 %,81.38 %
Ford High School,80.63 %,81.26 %,80.40 %,80.66 %
Griffin High School,83.37 %,83.71 %,84.29 %,84.01 %
Hernandez High School,80.87 %,80.66 %,81.40 %,80.86 %
Holden High School,83.68 %,83.32 %,83.82 %,84.70 %
Huang High School,81.29 %,81.51 %,81.42 %,80.31 %
Johnson High School,81.26 %,80.77 %,80.62 %,81.23 %


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# Create bins in which to place values based upon Budget per student
#Bins are created by "equal intervals" methodS
# Create labels for these bins
ngroups=4
bins=[]
group=[]
for i in range(ngroups+1):
    bins.append(min(SchoolSummary["Per Student Budget"])+((max(SchoolSummary["Per Student Budget"])-min(SchoolSummary["Per Student Budget"]))/ngroups)*i)
    if (i!=0):
        group.append(str("$ "+ str(bins[i-1])+" - "+str(bins[i])))
print(bins)
print(group)

[578.0, 597.25, 616.5, 635.75, 655.0]
['$ 578.0 - 597.25', '$ 597.25 - 616.5', '$ 616.5 - 635.75', '$ 635.75 - 655.0']


In [13]:
# Aply bins and labels to the data frame
#Group average data within the created bins
SchoolSummary["Spending Ranges (Per Student)"] = pd.cut(SchoolSummary["Per Student Budget"], bins, labels=group)
SchoolSummary.head()
SchoolSummary.groupby("Spending Ranges (Per Student)")['Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading',
       '% Overall Passing Rate'].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
$ 578.0 - 597.25,83.515798,83.915256,93.324222,96.634622,94.979422
$ 597.25 - 616.5,83.599686,83.885211,94.230858,95.900287,95.065572
$ 616.5 - 635.75,80.199966,82.425360,80.036217,89.536122,84.786170
$ 635.75 - 655.0,77.866721,81.368774,70.347325,82.995575,76.671450


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# Sample bins.
bins = [0, 1000, 2000, 5000]
group = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [15]:
# Aply bins and labels to the data frame
#Group average data within the created bins in to a data frame
SchoolSummary["School Size"] = pd.cut(SchoolSummary["Size"], bins, labels=group)
SchoolSummary.head()
SchoolSummary.groupby("School Size")['Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading',
       '% Overall Passing Rate'].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [17]:
#Group average data by "School Type" variable, in to a data frame
SchoolSummary.groupby("School Type")['Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading',
       '% Overall Passing Rate'].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [25]:
#Group average data by "School Type" and "School Size" variables, in to a data frame
SchoolSummary.groupby(["School Type","School Size"])['Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading',
       '% Overall Passing Rate'].mean()

Average Math Score  Average Reading Score  \
School Type School Size                                                     
Charter     Small (<1000)                83.821598              83.929843   
            Medium (1000-2000)           83.374684              83.864438   
            Large (2000-5000)            83.274201              83.989488   
District    Small (<1000)                      NaN                    NaN   
            Medium (1000-2000)                 NaN                    NaN   
            Large (2000-5000)            76.956733              80.966636   

                                % Passing Math  % Passing Reading  \
School Type School Size                                             
Charter     Small (<1000)            93.550225          96.099437   
            Medium (1000-2000)       93.599695          96.790680   
            Large (2000-5000)        93.867718          96.539641   
District    Small (<1000)                  NaN                NaN   
            Medium (1000-2000)             NaN                NaN   
            Large (2000-5000)        66.548453          80.799062   

                                % Overall Passing Rate  
School Type School Size                                 
Charter     Small (<1000)                    94.824831  
            Medium (1000-2000)               95.195187  
            Large (2000-5000)                95.203679  
District    Small (<1000)                          NaN  
            Medium (1000-2000)                     NaN  
            Large (2000-5000)                73.673757